In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import tqdm as tq
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor
from tqdm import tqdm

# 데이터 탐색

In [2]:
train = pd.read_csv('/content/drive/MyDrive/contest/parking_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/contest/parking_data/test.csv')
age_gender = pd.read_csv('/content/drive/MyDrive/contest/parking_data/age_gender_info.csv')

### 훈련 데이터

In [3]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [4]:
train.corr(method='pearson')

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
총세대수,1.000000,0.042271,0.098205,0.068192,0.213843,-0.001446,0.501488,0.316703
전용면적,0.042271,1.000000,-0.035479,-0.089201,-0.013111,-0.007866,0.071260,0.115453
전용면적별세대수,0.098205,-0.035479,1.000000,0.166472,-0.056680,0.047112,0.281989,0.247085
공가수,0.068192,-0.089201,0.166472,1.000000,-0.093334,0.043285,0.289819,0.118121
도보 10분거리 내 지하철역 수(환승노선 수 반영),0.213843,-0.013111,-0.056680,-0.093334,1.000000,0.098380,-0.055544,-0.108592
도보 10분거리 내 버스정류장 수,-0.001446,-0.007866,0.047112,0.043285,0.098380,1.000000,0.070696,0.103752
단지내주차면수,0.501488,0.071260,0.281989,0.289819,-0.055544,0.070696,1.000000,0.859216
등록차량수,0.316703,0.115453,0.247085,0.118121,-0.108592,0.103752,0.859216,1.000000


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

< Train data null 값 >

임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수

In [6]:
train.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,0
임대보증금,569


### 테스트 데이터

In [7]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

In [9]:
test.describe()

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
count,1022.000000,1022.000000,1022.000000,1022.000000,980.000000,1022.000000,1022.000000
mean,862.080235,43.706311,100.414873,15.544031,0.136735,4.626223,548.771037
std,536.340894,35.890759,125.997855,11.070140,0.435500,5.414568,342.636703
min,75.000000,9.960000,1.000000,0.000000,0.000000,1.000000,29.000000
25%,488.000000,33.135000,14.000000,6.000000,0.000000,2.000000,286.000000
50%,745.000000,39.720000,60.000000,15.000000,0.000000,3.000000,458.000000
75%,1161.000000,47.400000,140.000000,23.000000,0.000000,5.000000,711.000000
max,2572.000000,583.400000,1341.000000,45.000000,2.000000,50.000000,1696.000000


In [10]:
test.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,2
임대보증금,180


< Test data null 값 >

자격유형, 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영)

Train, Test null 값을 보면

임대보증금, 임대료는 같이 null값으로 존재할 것으로 생각할 수 있음

# 데이터 전처리

[ https://dacon.io/competitions/official/235745/codeshare/2853?page=1&dtype=recent ] 참고


임대가 아닌 공공분양, 전세 같은 경우는 임대 관련 값은 NA

임대보증금, 임대료를 0으로 대체

In [11]:
# 임대관련 값 '-'을 null로 바꾼 후 float로 타입 변경
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

# 임대관련 값 0으로 대체 
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

임대보증금, 임대료 dtype을 int로 변경

지하철 및 버스정류장 수 관련 Null 값도 없는 것으로 추정하고 0으로 대체

In [12]:
# 지하철 및 버스정류장 수 0으로 대체
col = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[col] = train[col].fillna(0)
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

test data의 자격유형은 최빈값으로 대체 

==> 'A'

In [13]:
test['자격유형'].value_counts() # A가 가장 많음을 확인

A    572
D    180
H     92
J     84
C     34
K     16
L     12
N     10
E     10
I      7
M      2
G      1
Name: 자격유형, dtype: int64

In [14]:
# A로 대체
test['자격유형'] = test['자격유형'].fillna('A')

중복이 상당히 많은 것으로 확인되므로 중복을 제거

In [15]:
train = train.drop_duplicates()
test = test.drop_duplicates()

[ https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent ]

train, test의 각 단지코드를 하나의 로우로 합하기

In [16]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

In [17]:
unique_cols = ['총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [18]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [19]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

In [20]:
train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [21]:
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

# 모델링

In [22]:
target_col = '등록차량수'
x = train_agg.drop(columns=[target_col, '지역'], axis = 1)
y = train_agg[target_col]

In [23]:
!pip install optuna

     |████████████████████████████████| 307kB 8.7MB/s 
     |████████████████████████████████| 174kB 15.6MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 112kB 18.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 143kB 18.5MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=2fdc5e92d7f040c4a7697a744e80c73c09134a609c3c4afc0c017163d77d149e
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eta=0.01, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1

In [24]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective(trial: Trial,x,y):
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.15,random_state=42)
    param = {
        'tree_method':'gpu_hist',
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'gamma' : trial.suggest_int('gamma', 0, 3),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.04, 0.06, 0.08,0.1]),
        "n_estimators" : trial.suggest_int('n_estimators', 100, 600),
        'max_depth': trial.suggest_categorical('max_depth', [3, 5,7,9,11,13,15,17,20]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    mae = mean_absolute_error(test_y, preds)
    
    return mae

In [27]:
study = optuna.create_study(direction='minimize', sampler =TPESampler())
study.optimize(lambda trial : objective(trial, x, y), n_trials=1000)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print(study.best_trial.value)

[I 2021-07-18 07:12:30,697] A new study created in memory with name: no-name-cf45fe7e-100a-4f8d-aa2b-046f68376364


[07:12:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-07-18 07:12:35,126] Trial 0 finished with value: 283.49515533447266 and parameters: {'alpha': 0.0012129912023808183, 'lambda': 0.007191432844172109, 'gamma': 1, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 460, 'max_depth': 20, 'min_child_weight': 235}. Best is trial 0 with value: 283.49515533447266.


[07:12:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-07-18 07:12:35,794] Trial 1 finished with value: 283.45594024658203 and parameters: {'alpha': 0.28185060291719805, 'lambda': 0.513896992149697, 'gamma': 3, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.04, 'n_estimators': 565, 'max_depth': 17, 'min_child_weight': 290}. Best is trial 1 with value: 283.45594024658203.


...

[07:25:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-07-18 07:25:48,974] Trial 998 finished with value: 141.57583022117615 and parameters: {'alpha': 0.7012755294840969, 'lambda': 2.3434670719250073, 'gamma': 0, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.08, 'n_estimators': 122, 'max_depth': 17, 'min_child_weight': 11}. Best is trial 996 with value: 127.3814360499382.


[07:25:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-07-18 07:25:50,657] Trial 999 finished with value: 127.97387206554413 and parameters: {'alpha': 0.8669727280621017, 'lambda': 8.671447198146323, 'gamma': 0, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.08, 'n_estimators': 142, 'max_depth': 17, 'min_child_weight': 1}. Best is trial 996 with value: 127.3814360499382.


Number of finished trials: 1000
Best trial: {'alpha': 0.8524947191279225, 'lambda': 2.871836224406721, 'gamma': 0, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.08, 'n_estimators': 139, 'max_depth': 17, 'min_child_weight': 1}
127.3814360499382


제출 파일 만들기

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/contest/parking_data/sample_submission.csv')

In [ ]:
predictions1 = catb.predict(test_agg)
predictions2 = xgb.predict(xgb_t_train)

In [ ]:
predictions = (predictions1 + predictions2) / 2
predictions

array([ 641.47461364, 1287.43615828,  569.0239828 ,  556.96060974,
       1035.53765716, 1860.12114562,  945.68167614,  423.8620673 ,
        352.11344383,  305.808364  ,  472.16747218,  229.58488984,
        361.03110274,  296.40333451,  401.72307259,  199.97980544,
        447.11667807,  278.57064989,  115.09329085,  719.79930553,
        194.51489326,  385.82775422,  461.01715726,  516.71176665,
        366.86272523,  147.79061541,   49.83104964,  494.39910262,
        367.21439604,  453.60971941,  810.6079746 ,  115.50925711,
        452.90621744,  210.12741572,   74.62601342,  297.47427924,
        462.74123204,  596.19028034,  788.23672694,  330.35941032,
        447.00028977,  409.6450722 ,  396.01550928,  516.19978799,
        694.22386651, 1261.04027791,  464.92236811,  599.95155602,
        380.16016501,  367.50764526,  741.05873649,  299.42629972,
       1028.11272719,  667.43600861,  576.13016643,  261.34224769,
        541.77235225,  316.38181096,  446.19100732,  163.86411

In [ ]:
sample_submission['num'] = predictions
sample_submission.head()

,code,num
0,C1072,641.474614
1,C1128,1287.436158
2,C1456,569.023983
3,C1840,556.960610
4,C1332,1035.537657


In [ ]:
file_name = 'parking06.csv'
sample_submission.to_csv('/content/drive/MyDrive/contest/parking_data/' + file_name, index=False)